In [1]:
import os
from pathlib import Path

In [5]:
template_config = '''out_dir: {out_dir}
metric_best: accuracy
device: 'cuda:0'
wandb:
  use: True
  project: {wandb_proj_name}
  entity: <WANDB_USERNAME>
dataset:
  format: PyG
  name: {ds}
  dir: {ds_download_dir}
  task: node
  task_type: classification_multi
  transductive: True
  split_mode: standard
  node_encoder: True
  node_encoder_name: LapPE
  node_encoder_bn: False
  edge_encoder: False
  edge_encoder_name: DummyEdge
  edge_encoder_bn: False
posenc_LapPE:
  enable: True
  eigen:
    laplacian_norm: none
    eigvec_norm: L2
    max_freqs: 4
  model: DeepSet
  dim_pe: 4
  layers: 2
  n_heads: 4  # Only used when `posenc.model: Transformer`
  raw_norm_type: none
train:
  mode: custom
  sampler: full_batch
  eval_period: 1
  enable_ckpt: False
model:
  type: GPSModel
  loss_fun: cross_entropy
  edge_decoding: dot
gt:
  layer_type: GCN+Transformer
  layers: {num_layers}
  n_heads: {num_heads}
  dim_hidden: {dim_hidden}  # `gt.dim_hidden` must match `gnn.dim_inner`
  dropout: {dropout}
  attn_dropout: {attn_dropout}
  layer_norm: False
  batch_norm: False
gnn:
  head: node
  layers_pre_mp: 0
  layers_post_mp: 1
  dim_inner: {dim_hidden}  # `gt.dim_hidden` must match `gnn.dim_inner`
  batchnorm: True
  act: gelu
  dropout: {dropout}
  agg: mean
  normalize_adj: False
optim:
  clip_grad_norm: True
  optimizer: adamW
  weight_decay: 1e-10
  base_lr: 0.001
  max_epoch: 1000
  scheduler: reduce_on_plateau
  num_warmup_epochs: 0
  early_stopping_patience: 30
  reduce_factor: 0.5
  schedule_patience: 15
share:
  dim_out: {num_classes}'''

In [6]:
NUM_CLASSES = {
    'Cora': 7,
    'CiteSeer': 6
}

## Make sure to add your correct paths!

In [7]:
for ds in ['Cora', 'CiteSeer']:
    num_classes = NUM_CLASSES[ds]

    for seed in [0]:
        for dim_hidden in [128]:
            for num_layers in [4]:
                for num_heads in [4]:
                    for attn_dropout in [0.1]:
                        for dropout in [0.0]:
                            CONFIG_PATH = f'<YOUR_PATH>/training_configs_{ds}'
                            SCRIPT_PATH = f'<YOUR_PATH>/training_scripts_{ds}'

                            Path(CONFIG_PATH).mkdir(exist_ok=True, parents=True)
                            Path(SCRIPT_PATH).mkdir(exist_ok=True, parents=True)
                        
                            dl_dir = f'<DL_PATH>/{ds}'
                            out_dir = f'<OUT_PATH>/{ds}/{seed}/{dim_hidden}/{num_layers}/{num_heads}/{attn_dropout}/{dropout}'
                            wandb_proj_name = f'<WANDB_PROJ>'

                            conf = template_config.format(
                                ds=ds, seed=seed, out_dir=out_dir, dim_hidden=dim_hidden, num_layers=num_layers, num_heads=num_heads,
                                ds_download_dir=dl_dir, wandb_proj_name=wandb_proj_name, num_classes=num_classes, attn_dropout=attn_dropout,
                                dropout=dropout
                            )
                            with open(os.path.join(CONFIG_PATH, f'{ds}_{seed}_{dim_hidden}_{num_layers}_{num_heads}_{attn_dropout}_{dropout}.yaml'), 'w') as f:
                                f.write(conf)

                            script = f'python <YOUR_PATH>/graphgps_node/main.py --cfg {CONFIG_PATH}/{ds}_{seed}_{dim_hidden}_{num_layers}_{num_heads}_{attn_dropout}_{dropout}.yaml'
                            with open(f'{SCRIPT_PATH}/{ds}_{seed}_{dim_hidden}_{num_layers}_{num_heads}_{attn_dropout}_{dropout}.sh', 'w') as f:
                                f.write(script)